<a href="https://colab.research.google.com/github/sandorkonya/VQA/blob/main/Procedural_creation_of_a_VQA_dataset_and_training_with_PaddleOCR_KIE_LayoutXML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#KIE training based on:
https://github.com/PaddlePaddle/PaddleOCR/blob/release/2.6/doc/doc_en/kie_en.md

In [ ]:
!git clone https://github.com/PaddlePaddle/PaddleOCR.git
%cd /content/PaddleOCR
!pip install -r requirements.txt

#issues with Cuda, 2.3.4 does not work
#https://github.com/open-mmlab/mmsegmentation/issues/943
#https://github.com/PaddlePaddle/Paddle/issues/46377
!pip install paddlepaddle-gpu==2.3.2
!pip install paddlenlp==2.4.4
!pip install yacs

import cv2
import numpy as np
import matplotlib.pyplot as plt
import random, time, datetime
import json
import os
from matplotlib.pyplot import figure

Cloning into 'PaddleOCR'...
remote: Enumerating objects: 45108, done.
remote: Counting objects: 100% (239/239), done.
remote: Compressing objects: 100% (166/166), done.
remote: Total 45108 (delta 107), reused 162 (delta 72), pack-reused 44869
Receiving objects: 100% (45108/45108), 338.39 MiB | 32.42 MiB/s, done.
Resolving deltas: 100% (31805/31805), done.
/content/PaddleOCR
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 619 kB 7.7 MB/s 
     |████████████████████████████████| 4.9 MB 72.8 MB/s 
     |████████████████████████████████| 2.2 MB 67.9 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 8.8 MB 54.9 MB/s 
     |████████████████████████████████| 132 kB 87.3 MB/s 
     |████████████████████████████████| 204 kB 95.9 MB/s 
     |████████████████████████████████| 399 kB 78.0

##generating training data

In [ ]:
%cd /content/PaddleOCR
%pwd
!mkdir train_data
%cd train_data


/content/PaddleOCR
/content/PaddleOCR/train_data


In [ ]:
def answer(q):
    if q == "Name:":
        return  random.choice(names);
    if q == "DOB:":
        datestring = datetime.datetime.strftime(datetime.datetime( \
    random.randint(2000, 2015), \
    random.randint(1, 12), \
    random.randint(1, 28), \
    random.randrange(23), \
    random.randrange(59), \
    random.randrange(59), \
    random.randrange(1000000)), '%Y-%m-%d')
        return datestring;
    if q == "Birthplace:":
        return random.choice(city_list)[:13];
    if q == "Animal:":
        return random.choice(animals)[:13];
    if q == "ID:":
        return "abc" + str(random.randint(724, 999)) + str(random.randint(0, 9)) + str(random.randint(100, 999)) + "A" + str(random.randint(10000, 99999))       
    return q

names = ["Michael","Christopher","Jessica","Matthew","Ashley","Jennifer","Joshua","Amanda","Daniel","David","James","Robert","John","Joseph","Andrew","Ryan","Brandon","Jason","Justin","Sarah","William","Jonathan","Stephanie","Brian","Nicole","Nicholas","Anthony","Heather","Eric","Elizabeth","Adam","Megan","Melissa","Kevin","Steven","Thomas","Timothy","Christina","Kyle","Rachel","Laura","Lauren","Amber","Brittany","Danielle","Richard","Kimberly","Jeffrey","Amy","Crystal","Michelle","Tiffany","Jeremy","Benjamin","Mark","Emily","Aaron","Charles","Rebecca","Jacob","Stephen","Patrick","Sean","Erin","Zachary","Jamie","Kelly","Samantha","Nathan","Sara","Dustin","Paul","Angela","Tyler","Scott","Katherine","Andrea","Gregory","Erica","Mary","Travis","Lisa","Kenneth","Bryan","Lindsey","Kristen","Jose","Alexander","Jesse","Katie","Lindsay","Shannon","Vanessa","Courtney","Christine","Alicia","Cody","Allison","Bradley","Samuel","Shawn","April","Derek","Kathryn","Kristin","Chad","Jenna","Tara","Maria","Krystal","Jared","Anna","Edward","Julie","Peter","Holly","Marcus","Kristina","Natalie","Jordan","Victoria","Jacqueline","Corey","Keith","Monica","Juan","Donald","Cassandra","Meghan","Joel","Shane","Phillip","Patricia","Brett","Ronald","Catherine","George","Antonio","Cynthia","Stacy","Kathleen","Raymond","Carlos","Brandi","Douglas","Nathaniel","Ian","Craig","Brandy","Alex","Valerie","Veronica","Cory","Whitney","Gary","Derrick","Philip","Luis","Diana","Chelsea","Leslie","Caitlin","Leah","Natasha","Erika","Casey","Latoya","Erik","Dana","Victor","Brent","Dominique","Frank","Brittney","Evan","Gabriel","Julia","Candice","Karen","Melanie","Adrian","Stacey","Margaret","Sheena","Wesley","Vincent","Alexandra","Katrina","Bethany","Nichole","Larry","Jeffery","Curtis","Carrie","Todd","Blake","Christian","Randy","Dennis","Alison","Trevor","Seth","Kara","Joanna","Rachael","Luke","Felicia","Brooke","Austin","Candace","Jasmine","Jesus","Alan","Susan","Sandra","Tracy","Kayla","Nancy","Tina","Krystle","Russell","Jeremiah","Carl","Miguel","Tony","Alexis","Gina","Jillian","Pamela","Mitchell","Hannah","Renee","Denise","Molly","Jerry","Misty","Mario","Johnathan","Jaclyn","Brenda","Terry","Lacey","Shaun","Devin","Heidi","Troy","Lucas","Desiree","Jorge","Andre","Morgan","Drew","Sabrina","Miranda","Alyssa","Alisha","Teresa","Johnny","Meagan","Allen","Krista","Marc","Tabitha","Lance","Ricardo","Martin","Chase"]
city_list = ["Kyiv","Kharkiv","Donetsk","Odessa","Dnipro","Zaporizhzhya","Lviv","Kryvyy Rih","Mykolayiv","Sevastopol","Luhansk","Mariupol","Makiivka","Vinnytsya","Simferopol","Chernihiv","Sumy","Kherson","Poltava","Horlivka","Cherkasy","Khmelnytskyi","Chernivtsi","Zhytomyr","Rivne","Ivano-Frankivsk","Kamyanske","Kropyvnytskyy","Ternopil","Kremenchuk","Lutsk","Bila Tserkva","Melitopol","Kramatorsk","Kerch","Sievierodonetsk","Alchevsk","Uzhgorod","Nikopol","Berdyansk","Sloviansk","Yevpatoriya","Pavlohrad","Kamyianets-Podilskyy","Lysychansk","Konotop","Kostiantynivka","Krasnyi Luch","Brovary","Berdychiv","Mukachevo","Shostka","Kadiyivka","Uman","Chervonohrad","Bakhmut","Drohobych","Oleksandriya","Yenakiieve","Yalta","Nizhyn","Shakhtarsk","Izmail","Novomoskovsk","Feodosiya","Pokrovsk","Dovzhansk","Smila","Kalush","Kovel","Korosten","Pervomaysk","Pryluky","Irpin","Druzhkivka","Lozova","Stryi","Kolomyya","Chornomors'k","Khartsyzk","Rubizhne","Novohrad-Volynskyi","Snizhne","Boryspil","Izium","Chystyakove","Enerhodar","Antratsyt","Fastiv","Novovolynsk","Horishni Plavni","Bilhorod-Dnistrovskyi","Okhtyrka","Myrnohrad","Sorokyne","Romny","Marhanets","Rovenky","Lubny","Nova Kakhovka","Brianka","Svitlovodsk","Zhovti Vody","Vyshneve","Bucha","Varash","Toretsk","Dzhankoi","Shepetivka","Podilsk","Myrhorod","Yuzhnoukrayinsk","Volodymyr-Volynskyi","Pokrov","Dubno","Yasynuvata","Vasylkiv","Netishyn","Chuhuiv","Pervomaisk","Kakhovka","Avdiyivka","Slavuta","Boiarka","Sambir","Voznesensk","Starokostyantyniv","Zhmerynka","Boryslav","Yuzhne","Kupiansk","Hlukhiv","Obukhiv","Novoyavorivs'k","Kostopil","Tokmak","Yany Kapu","Mohyliv-Podilskyy","Synelnykove","Alushta","Perevalsk","Pervomaiskyi","Chortkiv","Dobropillia","Truskavets","Khust","Saki","Ternivka","Zolotonosha","Khmilnyk","Pershotravensk","Khrestivka","Bakhchysarai","Sarny","Balakliia","Holubivka","Pereiaslav","Haysyn","Malyn","Vynohradiv","Popasna","Slavutych","Zdolbuniv","Oleshky","Korostyshiv","Armyansk","Debaltseve","Lebedyn","Krolevets","Kaniv","Berehove","Volnovakha","Hadyach","Brody","Kozyatyn","Dokuchaievsk","Ladyzhyn","Zolochiv","Molodohvardiisk","Vilnohirsk","Nadvirna","Znamyanka","Vyshhorod","Starobilsk","Selydove","Krasnohrad","Merefa","Kremenets","Polonne","Kurakhovo","Dolyna","Sokal","Lyman","Stebnyk","Lyubotyn","Kivsharivka","Kiliya","Henichesk","Vovchansk","Pidhorodne","Yahotyn","Kalynivka","Polohy","Krasyliv","Volochysk","Svatove","Balaklava","Reni","Balta","Kurakhove","Dolynska","Pyatykhatky","Kreminna","Amvrosiivka","Dniprorudne","Zuhres","Skadovsk","Rozdilna","Bohodukhiv","Korsun-Shevchenkivskyy","Berezhany","Hostomel","Skvyra","Derhachi","Svalyava","Bakhmach","Lutuhyne","Zvenihorodka","Shpola","Sudak","Izyaslav","Bohuslav","Novoukrayinka","Bilohirsk","Berezan","Verkhnodniprovsk","Vatutine","Horodok","Horodok","Bar","Bilopillia","Apostolove","Slobozhans'ke","Dunayivtsi","Rakhiv","Ilovays'k","Pyriatyn","Ovruch","Chervonopartyzansk","Radomyshl","Putyvl","Vilnyansk","Bolhrad","Tulchyn","Novyy Buh","Krasnohorivka","Burshtyn","Artsyz","Hayvoron","Orikhiv","Karlivka","Novhorod-Siverskyi","Tetiiv","Nyzhnia Krynka","Ostroh","Mykolaiv","Koriukivka","Storozhynets","Ochakiv","Zmiiv","Hola Prystan","Kotsiubynske","Kivertsi","Velykodolynske","Zbarazh","Ukrainka","Zhashkiv","Horodyshche","Slobozhanske","Nova Odesa","Bunhe","Hulyaypole","Terebovlya","Talne","Rozhyshche","Kaharlyk","Vasylivka","Nosivka","Lymanka","Yavoriv","Zhovkva","Tarashcha","Khorol","Antonivka","Perehinske","Uzyn","Petrovske","Bershad","Kamyanka-Dniprovska","Hvardiiske","Primorskiy","Chasiv Yar","Bashtanka","Stanytsia Luhanska","Snihurivka","Bilyayivka","Beryslav","Buchach","Solonytsivka","Hnivan","Svetlodarsk","Kotelva","Borodianka","Mykhaylivka","Rokytne","Horodnia","Prymorsk","Nemyriv","Ovidiopol","Shchastia","Kostiantynivka","Kamyanka","Yakymivka","Kal'mius'ke","Baranivka","Lokhvytsya","Illintsi","Myronivka","Krasnogvardeyskoye","Novomyrhorod","Novoazovs'k","Mena","Siversk","Kamin-Kashyrskyi","Chernomorskoye","Tatarbunary","Vysokyi","Yampil","Borshchiv","Ukrainsk","Snovsk","Baryshivka","Bobrynets","Bobrovytsia","Hrebinka","Soledar","Novotroyitske","Nova Vodolaha","Shchyolkino","Novodnistrovsk","Ichnia","Sartana","Mospyne","Bolekhiv","Radyvyliv","Khrystynivka","Krasnoyilsk","Liuboml","Sovetskiy","Mala Vyska","Valky","Letychiv","Gaspra","Verkhivtseve","Tavriysk","Oktyabr'skoye","Inkerman","Ivankiv","Korolevo","Novooleksiyivka","Derazhnya","Teplodar","Shyroke","Pokotylivka","Rodynske","Sniatyn","Dubove","Mar'yinka","Pokrovske","Gresovskiy","Dubliany","Kobelyaky","Radekhiv","Novopskov","Pereshchepyne","Vesele","Chaplynka","Ratne","Berezivka","Borzna","Pohrebyshche","Velyka Dymerka","Zymohiria","Makariv","Mizhhirya","Bilozerka","Smoline","Zinkiv","Novyi Svit","Pustomyty"]
animals = ["Aardvark","Aardwolf","Abyssinian","Abyssinian Guinea Pig","Acadian Flycatcher","Achrioptera Manga","Ackie Monitor","Addax","Adelie Penguin","Aesculapian snake","Affenpinscher","Afghan Hound","African Bullfrog","African Bush Elephant","African Civet","African Clawed Frog","African Fish Eagle","African Forest Elephant","African Golden Cat","African Grey Parrot","African Jacana","African Palm Civet","African Penguin","African Tree Toad","African Wild Dog","Africanized bee (killer bee)","Agama Lizard","Aidi","Ainu","Airedale Terrier","Airedoodle","Akbash","Akita","Akita Shepherd","Alabai (Central Asian Shepherd)","Alaskan Husky","Alaskan Klee Kai","Alaskan Malamute","Alaskan Pollock","Alaskan Shepherd","Albacore Tuna","Albatross","Albertonectes","Albino (Amelanistic) Corn Snake","Aldabra Giant Tortoise","Alligator Gar","Allosaurus","Allosaurus","Alpaca","Alpine Dachsbracke","Alpine Goat","Alusky","Amano Shrimp","Amargasaurus","Amazon Parrot","Amazon River Dolphin (Pink Dolphin)","Amazon Tree Boa","Amazonian Royal Flycatcher","Ambrosia Beetle","American Alligator","American Alsatian","American Bulldog","American Bully","American Cocker Spaniel","American Cockroach","American Coonhound","American Dog Tick","American Eel","American Eskimo Dog","American Foxhound","American Hairless Terrier","American Leopard Hound","American Paddlefish","American Pit Bull Terrier","American Pugabull","American Pygmy Goat","American Robin","American Staffordshire Terrier","American Toad","American Water Spaniel","American Wirehair","Amethystine Python (Scrub Python)","Amphicoelias Fragillimus","Amur Leopard","Anaconda","Anatolian Shepherd Dog","Anchovies","Andrewsarchus","Angelfish","Angelshark","Angled Sunbeam Caterpillar","Anglerfish","Angora Ferret","Angora Goat","Anhinga","Anna’s Hummingbird","Anole Lizard","Anomalocaris","Ant","Antarctic scale worm","Anteater","Antelope","Anteosaurus","Ape","Apennine Wolf","Appenzeller Dog","Apple Head Chihuahua","Apple Moth","Arabian Cobra","Arafura File Snake","Arapaima","Archaeoindris","Archaeopteryx","Archaeotherium","Archelon Turtle","Archerfish","Arctic Char","Arctic Fox","Arctic Hare","Arctic Wolf","Arctodus","Arctotherium","Argentavis Magnificens","Argentinosaurus","Arizona Bark Scorpion","Arizona Black Rattlesnake","Arizona Blonde Tarantula","Arizona Coral Snake","Armadillo","Armadillo Lizard","Armenian Gampr","Armored Catfish","Armyworm","Arsinoitherium","Arthropleura","Aruba Rattlesnake","Asian Arowana","Asian Carp","Asian Cockroach","Asian Elephant","Asian Giant Hornet","Asian Lady Beetle","Asian Longhorn Beetle","Asian Palm Civet","Asian Vine Snake","Asian Water Monitor","Asiatic Black Bear","Asp","Assassin Bug","Atlantic Salmon","Atlantic Sturgeon","Atlas Beetle","Atlas Moth","Aurochs","Aussiedoodle","Aussiedor","Aussiepom","Australian Bulldog","Australian Cattle Dog","Australian Cockroach","Australian Flathead Perch","Australian Gecko","Australian Kelpie Dog","Australian Labradoodle","Australian Mist","Australian Retriever","Australian Shepherd","Australian Terrier","Australopithecus","Avocet","Axanthic Ball Python","Axolotl","Ayam Cemani","Aye-aye"]



In [ ]:
questions = ["Name:",  "DOB:", "Birthplace:", "Animal:", "ID:"]
question_labels = ["ANSWER_NAME",  "ANSWER_DOB", "ANSWER_BIRTHPLACE", "ANSWER_ANIMAL", "ANSWER_ID"]

path = "/content/PaddleOCR/train_data/sample/"

if not os.path.exists(path):
  %cd /content/PaddleOCR
  !mkdir sample
  os.makedirs(path)

trainfolder = path + "train/"
valfolder = path + "validation/"

if not os.path.exists(trainfolder):
   os.makedirs(trainfolder)
if not os.path.exists(valfolder):
   os.makedirs(valfolder)

# save class.txt
classes = "QUESTION" + "\n" + "\n".join(question_labels) + "\nOTHER"

with open(path + "class_list.txt", 'w+') as fout:    
    print(classes, file=fout)

filenumber = 250

for n in range(0,filenumber):
    labels = []

    img = 255 * np.ones(shape=[750,500, 3], dtype=np.uint8)
    k = 0

    for i in questions:

        (wq, hq), _ = cv2.getTextSize(
                i, cv2.FONT_HERSHEY_SIMPLEX, 0.6, 1)

        x1 = 50 
        y1 = 100 + 50 * k

        # Prints the text.    
        #img = cv2.rectangle(img, (x1, y1 - 2*hq), (x1 + int(1.7*wq), y1), (0,255,0), -1)
        img = cv2.putText(img, i, (x1, y1 - 5),
                    fontFace= cv2.FONT_HERSHEY_SIMPLEX, fontScale=1, color=(0,0,0),
                    thickness=1, lineType=cv2.LINE_AA)
        
        tl = [x1, y1 - 2*hq]
        tr = [x1 + int(1.7*wq), y1 - 2*hq]
        bl = [x1, y1]
        br = [x1 + int(1.7*wq), y1]
        
        d = { "transcription": i,
                         "label": "QUESTION",
                         "points": [bl, br, tr, tl],
                         "id" :"Q_" + str(n) + str(k),
                        "linking": []
            }

        labels.append(d)
        
        
        ans  = answer(i)
        
        (wa, ha), _ = cv2.getTextSize(
                ans, cv2.FONT_HERSHEY_SIMPLEX, 0.6, 1)

        #img = cv2.rectangle(img, (x1 + int(1.7*wq) + 20 , y1 - 2*ha), (x1 + int(1.7*wq) + int(1.7*wa) + 20, y1), (255,0,0), -1)
        img = cv2.putText(img, ans, (x1 + int(1.7*wq) + 20 , y1 - 5),
                    fontFace= cv2.FONT_HERSHEY_SIMPLEX, fontScale=1, color=(0,0,0),
                    thickness=1, lineType=cv2.LINE_AA)
        
        tl = [x1 + int(1.7*wq) + 20,  y1 - 2*ha]
        tr = [x1 + int(1.7*wq) + int(1.7*wa) + 20,  y1 - 2*ha]
        bl = [x1 + int(1.7*wq) + 20, y1]
        br = [x1 + int(1.7*wq) + int(1.7*wa) + 20, y1]
        
        d = { "transcription": ans,
                         "label": question_labels[k],
                         "points": [bl, br, tr, tl],
                         "id" : "A_" + str(n) + str(k),
                        "linking": []
            }
        
        k = k +1
        labels.append(d)
        
    #figure(figsize=(8, 6), dpi=80)
    #plt.imshow(img)
    #plt.show()

    j = json.dumps(labels)
    
    filename = answer("ID:")

    if n < int(filenumber*0.8) :
        cv2.imwrite(trainfolder + filename+".jpg", img)

        with open(path + "training.json", 'a') as fout:    
            print(filename+ ".jpg" + "\t"+j, file=fout)
    else:
        cv2.imwrite(valfolder + filename+".jpg", img)

        with open(path + "val.json", 'a') as fout:    
            print(filename+ ".jpg"  + "\t"+j, file=fout)
            
    print(filename, j)


/content/PaddleOCR
abc7463671A37515 [{"transcription": "Name:", "label": "QUESTION", "points": [[50, 100], [152, 100], [152, 72], [50, 72]], "id": "Q_00", "linking": []}, {"transcription": "Jennifer", "label": "ANSWER_NAME", "points": [[172, 100], [299, 100], [299, 72], [172, 72]], "id": "A_00", "linking": []}, {"transcription": "DOB:", "label": "QUESTION", "points": [[50, 150], [126, 150], [126, 122], [50, 122]], "id": "Q_01", "linking": []}, {"transcription": "2006-05-06", "label": "ANSWER_DOB", "points": [[146, 150], [363, 150], [363, 122], [146, 122]], "id": "A_01", "linking": []}, {"transcription": "Birthplace:", "label": "QUESTION", "points": [[50, 200], [220, 200], [220, 172], [50, 172]], "id": "Q_02", "linking": []}, {"transcription": "Vysokyi", "label": "ANSWER_BIRTHPLACE", "points": [[240, 200], [355, 200], [355, 172], [240, 172]], "id": "A_02", "linking": []}, {"transcription": "Animal:", "label": "QUESTION", "points": [[50, 250], [165, 250], [165, 222], [50, 222]], "id": "Q

## config for the custom

for VI-LayoutXML training

In [ ]:
config =  """
Global:
  use_gpu: True
  epoch_num: &epoch_num 100
  log_smooth_window: 10
  print_batch_step: 10
  save_model_dir: ./output/sample
  save_epoch_step: 100
  # evaluation is run every 10 iterations after the 0th iteration
  eval_batch_step: [ 0, 10 ]
  cal_metric_during_train: False
  save_inference_dir: /content/PaddleOCR/output/ser/sample/res
  use_visualdl: True
  seed: 2022
  infer_img: /content/PaddleOCR/train_data/sample/validation/
  # if you want to predict using the groundtruth ocr info,
  # you can use the following config
  # infer_img: train_data/XFUND/zh_val/val.json
  # infer_mode: False

  save_res_path: /content/PaddleOCR/output/ser/sample/res

  kie_rec_model_dir:
  kie_det_model_dir:

Architecture:
  model_type: kie
  algorithm: &algorithm "LayoutXLM"
  Transform:
  Backbone:
    name: LayoutXLMForSer
    pretrained: True
    checkpoints:
    # one of base or vi
    mode: vi
    num_classes: &num_classes 13

Loss:
  name: VQASerTokenLayoutLMLoss
  num_classes: *num_classes
  key: "backbone_out"

Optimizer:
  name: AdamW
  beta1: 0.9
  beta2: 0.999
  lr:
    name: Linear
    learning_rate: 0.00005
    epochs: *epoch_num
    warmup_epoch: 2
  regularizer:
    name: L2
    factor: 0.00000

PostProcess:
  name: VQASerTokenLayoutLMPostProcess
  class_path: &class_path /content/PaddleOCR/train_data/sample/class_list.txt

Metric:
  name: VQASerTokenMetric
  main_indicator: hmean

Train:
  dataset:
    name: SimpleDataSet
    data_dir: /content/PaddleOCR/train_data/sample/train/
    label_file_list: 
      - /content/PaddleOCR/train_data/sample/training.json
    ratio_list: [ 1.0 ]
    transforms:
      - DecodeImage: # load image
          img_mode: RGB
          channel_first: False
      - VQATokenLabelEncode: # Class handling label
          contains_re: False
          algorithm: *algorithm
          class_path: *class_path
          use_textline_bbox_info: &use_textline_bbox_info True
          # one of [None, "tb-yx"]
          order_method: &order_method "tb-yx"
      - VQATokenPad:
          max_seq_len: &max_seq_len 512
          return_attention_mask: True
      - VQASerTokenChunk:
          max_seq_len: *max_seq_len
      - Resize:
          size: [224,224]
      - NormalizeImage:
          scale: 1
          mean: [ 123.675, 116.28, 103.53 ]
          std: [ 58.395, 57.12, 57.375 ]
          order: 'hwc'
      - ToCHWImage:
      - KeepKeys:
          keep_keys: ['input_ids', 'bbox', 'attention_mask', 'token_type_ids', 'image', 'labels'] # dataloader will return list in this order
  loader:
    shuffle: True
    drop_last: False
    batch_size_per_card: 8
    num_workers: 4

Eval:
  dataset:
    name: SimpleDataSet
    data_dir: /content/PaddleOCR/train_data/sample/validation/
    label_file_list:
      - /content/PaddleOCR/train_data/sample/val.json
    transforms:
      - DecodeImage: # load image
          img_mode: RGB
          channel_first: False
      - VQATokenLabelEncode: # Class handling label
          contains_re: False
          algorithm: *algorithm
          class_path: *class_path
          use_textline_bbox_info: *use_textline_bbox_info
          order_method: *order_method
      - VQATokenPad:
          max_seq_len: *max_seq_len
          return_attention_mask: True
      - VQASerTokenChunk:
          max_seq_len: *max_seq_len
      - Resize:
          size: [224,224]
      - NormalizeImage:
          scale: 1
          mean: [ 123.675, 116.28, 103.53 ]
          std: [ 58.395, 57.12, 57.375 ]
          order: 'hwc'
      - ToCHWImage:
      - KeepKeys:
          keep_keys: [ 'input_ids', 'bbox', 'attention_mask', 'token_type_ids', 'image', 'labels'] # dataloader will return list in this order
  loader:
    shuffle: False
    drop_last: False
    batch_size_per_card: 8
    num_workers: 4

"""

with open('/content/PaddleOCR/configs/kie/vi_layoutxlm/ser_vi_layoutxlm_sample.yml', 'w') as f:
    f.write(config)

##training

In [ ]:
%cd /content/PaddleOCR

!python3 tools/train.py -c /content/PaddleOCR/configs/kie/vi_layoutxlm/ser_vi_layoutxlm_sample.yml


/content/PaddleOCR
[2022/12/09 13:23:06] ppocr INFO: Architecture : 
[2022/12/09 13:23:06] ppocr INFO:     Backbone : 
[2022/12/09 13:23:06] ppocr INFO:         checkpoints : None
[2022/12/09 13:23:06] ppocr INFO:         mode : vi
[2022/12/09 13:23:06] ppocr INFO:         name : LayoutXLMForSer
[2022/12/09 13:23:06] ppocr INFO:         num_classes : 13
[2022/12/09 13:23:06] ppocr INFO:         pretrained : True
[2022/12/09 13:23:06] ppocr INFO:     Transform : None
[2022/12/09 13:23:06] ppocr INFO:     algorithm : LayoutXLM
[2022/12/09 13:23:06] ppocr INFO:     model_type : kie
[2022/12/09 13:23:06] ppocr INFO: Eval : 
[2022/12/09 13:23:06] ppocr INFO:     dataset : 
[2022/12/09 13:23:06] ppocr INFO:         data_dir : /content/PaddleOCR/train_data/sample/validation/
[2022/12/09 13:23:06] ppocr INFO:         label_file_list : ['/content/PaddleOCR/train_data/sample/val.json']
[2022/12/09 13:23:06] ppocr INFO:         name : SimpleDataSet
[2022/12/09 13:23:06] ppocr INFO:         transf

##evaluation

In [ ]:
%cd /content/PaddleOCR
!python3 tools/eval.py -c /content/PaddleOCR/configs/kie/vi_layoutxlm/ser_vi_layoutxlm_sample.yml -o Architecture.Backbone.checkpoints=/content/PaddleOCR/output/sample/best_accuracy


/content/PaddleOCR
[2022/12/09 13:32:35] ppocr INFO: Architecture : 
[2022/12/09 13:32:35] ppocr INFO:     Backbone : 
[2022/12/09 13:32:35] ppocr INFO:         checkpoints : /content/PaddleOCR/output/sample/best_accuracy
[2022/12/09 13:32:35] ppocr INFO:         mode : vi
[2022/12/09 13:32:35] ppocr INFO:         name : LayoutXLMForSer
[2022/12/09 13:32:35] ppocr INFO:         num_classes : 13
[2022/12/09 13:32:35] ppocr INFO:         pretrained : True
[2022/12/09 13:32:35] ppocr INFO:     Transform : None
[2022/12/09 13:32:35] ppocr INFO:     algorithm : LayoutXLM
[2022/12/09 13:32:35] ppocr INFO:     model_type : kie
[2022/12/09 13:32:35] ppocr INFO: Eval : 
[2022/12/09 13:32:35] ppocr INFO:     dataset : 
[2022/12/09 13:32:35] ppocr INFO:         data_dir : /content/PaddleOCR/train_data/sample/validation/
[2022/12/09 13:32:35] ppocr INFO:         label_file_list : ['/content/PaddleOCR/train_data/sample/val.json']
[2022/12/09 13:32:35] ppocr INFO:         name : SimpleDataSet
[2022/

##inference with newly trained checkpoint weights

In [ ]:
%cd /content/PaddleOCR/
!python3 /content/PaddleOCR/tools/infer_kie_token_ser.py \
 -c /content/PaddleOCR/configs/kie/vi_layoutxlm/ser_vi_layoutxlm_sample.yml \
 -o Architecture.Backbone.checkpoints=/content/PaddleOCR/output/sample/best_accuracy \
 Global.infer_img=/content/PaddleOCR/train_data/sample/validation/ \
 Global.use_gpu=False


/content/PaddleOCR
[2022/12/09 13:33:06] ppocr INFO: Architecture : 
[2022/12/09 13:33:06] ppocr INFO:     Backbone : 
[2022/12/09 13:33:06] ppocr INFO:         checkpoints : /content/PaddleOCR/output/sample/best_accuracy
[2022/12/09 13:33:06] ppocr INFO:         mode : vi
[2022/12/09 13:33:06] ppocr INFO:         name : LayoutXLMForSer
[2022/12/09 13:33:06] ppocr INFO:         num_classes : 13
[2022/12/09 13:33:06] ppocr INFO:         pretrained : True
[2022/12/09 13:33:06] ppocr INFO:     Transform : None
[2022/12/09 13:33:06] ppocr INFO:     algorithm : LayoutXLM
[2022/12/09 13:33:06] ppocr INFO:     model_type : kie
[2022/12/09 13:33:06] ppocr INFO: Eval : 
[2022/12/09 13:33:06] ppocr INFO:     dataset : 
[2022/12/09 13:33:06] ppocr INFO:         data_dir : /content/PaddleOCR/train_data/sample/validation/
[2022/12/09 13:33:06] ppocr INFO:         label_file_list : ['/content/PaddleOCR/train_data/sample/val.json']
[2022/12/09 13:33:06] ppocr INFO:         name : SimpleDataSet
[2022/

In [ ]:
!python /content/PaddleOCR/tools/export_model.py -c /content/PaddleOCR/configs/kie/vi_layoutxlm/ser_vi_layoutxlm_sample.yml

[2022-12-03 22:30:50,327] [    INFO] - Already cached /root/.paddlenlp/models/vi-layoutxlm-base-uncased/model_state.pdparams
W1203 22:30:50.328397  3686 gpu_resources.cc:61] Please NOTE: device: 0, GPU Compute Capability: 7.5, Driver API Version: 11.2, Runtime API Version: 10.2
W1203 22:30:50.346951  3686 gpu_resources.cc:91] device: 0, cuDNN Version: 8.1.
tcmalloc: large alloc 1073750016 bytes == 0xa93ee000 @  0x7fae39371887 0x7fae37cb907d 0x7fae37cacca8 0x7fae37cb74ab 0x7fae37ca7782 0x7fadd94c0855 0x7fadd579241a 0x7fadd2fbbf6e 0x7fadd2fbdb84 0x7fadd60b9be2 0x7fadd60a3730 0x7fadd2d7629b 0x7fadd2634f8e 0x7fadd26aa532 0x5d746e 0x5d813c 0x4ff515 0x49caa1 0x55e571 0x5d7cf1 0x49ca7c 0x4fdff5 0x49ced5 0x55e571 0x5d7cf1 0x49ca7c 0x55e571 0x5d7cf1 0x5d78b6 0x561051 0x55e858
tcmalloc: large alloc 2147491840 bytes == 0xe93f0000 @  0x7fae39371887 0x7fae37cb907d 0x7fae37cacca8 0x7fae37cb74ab 0x7fae37ca7782 0x7fadd94c0855 0x7fadd579241a 0x7fadd2fbbf6e 0x7fadd2fbdb84 0x7fadd60b9be2 0x7fadd60a3730 0

In [ ]:
%cd /content/PaddleOCR/ppstructure

!python3 kie/predict_kie_token_ser.py \
--kie_algorithm=LayoutXLM \
--ser_model_dir=/content/PaddleOCR/output/ser/sample/res/ \
--image_dir=/content/PaddleOCR/train_data/sample/val \
--ser_dict_path=/content/PaddleOCR/train_data/sample/class_list.txt \
--vis_font_path=/content/PaddleOCR/doc/fonts/latin.ttf \
--ocr_order_method="tb-yx" \
--rec_model_dir="./en_PP-OCRv3_rec_infer/" \
--det_model_dir="./en_PP-OCRv3_det_infer/" \
--use_gpu False

/content/PaddleOCR/ppstructure
download https://paddleocr.bj.bcebos.com/PP-OCRv3/chinese/ch_PP-OCRv3_det_infer.tar to ./en_PP-OCRv3_det_infer/ch_PP-OCRv3_det_infer.tar
100% 3.83M/3.83M [00:02<00:00, 1.83MiB/s]
download https://paddleocr.bj.bcebos.com/PP-OCRv3/chinese/ch_PP-OCRv3_rec_infer.tar to ./en_PP-OCRv3_rec_infer/ch_PP-OCRv3_rec_infer.tar
100% 11.9M/11.9M [00:10<00:00, 1.09MiB/s]
download https://paddleocr.bj.bcebos.com/dygraph_v2.0/ch/ch_ppocr_mobile_v2.0_cls_infer.tar to /root/.paddleocr/whl/cls/ch_ppocr_mobile_v2.0_cls_infer/ch_ppocr_mobile_v2.0_cls_infer.tar
100% 2.19M/2.19M [00:03<00:00, 595kiB/s]
[2022-12-06 16:31:32,214] [    INFO] - Already cached /root/.paddlenlp/models/layoutxlm-base-uncased/sentencepiece.bpe.model
[2022-12-06 16:31:32,668] [    INFO] - tokenizer config file saved in /root/.paddlenlp/models/layoutxlm-base-uncased/tokenizer_config.json
[2022-12-06 16:31:32,669] [    INFO] - Special tokens file saved in /root/.paddlenlp/models/layoutxlm-base-uncased/speci